## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
#from config import g_key
g_key = ""

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,29,Katobu,ID,76.08,broken clouds,-4.9408,122.5276,La Ode Husein
1,30,Boyolangu,ID,75.60,overcast clouds,-8.1181,111.8935,Yos ega cv
2,36,Rikitea,PF,78.69,broken clouds,-23.1203,-134.9692,People ThankYou
3,38,Atuona,PF,78.51,broken clouds,-9.8000,-139.0333,Villa Enata
4,40,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,29,Katobu,ID,76.08,broken clouds,-4.9408,122.5276,La Ode Husein
1,30,Boyolangu,ID,75.60,overcast clouds,-8.1181,111.8935,Yos ega cv
2,36,Rikitea,PF,78.69,broken clouds,-23.1203,-134.9692,People ThankYou
3,38,Atuona,PF,78.51,broken clouds,-9.8000,-139.0333,Villa Enata
4,40,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
5,44,Bengkulu,ID,77.09,moderate rain,-3.8004,102.2655,Grage Hotel Bengkulu
6,48,Hobyo,SO,78.71,overcast clouds,5.3505,48.5268,Halane Hotel and Restaurent
7,50,Pisco,PE,75.25,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
8,80,Iquique,CL,75.24,clear sky,-20.2208,-70.1431,Backpacker`s Hostel Iquique
9,82,Namatanai,PG,78.49,scattered clouds,-3.6667,152.4333,Seagulls Inn Namatani


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                85
City                   85
Country                85
Max Temp               85
Current Description    85
Lat                    85
Lng                    85
Hotel Name             85
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                85
City                   85
Country                85
Max Temp               85
Current Description    85
Lat                    85
Lng                    85
Hotel Name             85
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Katobu,ID,76.08,broken clouds,-4.9408,122.5276,
1,Boyolangu,ID,75.60,overcast clouds,-8.1181,111.8935,
2,Rikitea,PF,78.69,broken clouds,-23.1203,-134.9692,
3,Atuona,PF,78.51,broken clouds,-9.8000,-139.0333,
4,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
5,Bengkulu,ID,77.09,moderate rain,-3.8004,102.2655,
6,Hobyo,SO,78.71,overcast clouds,5.3505,48.5268,
7,Pisco,PE,75.25,clear sky,-13.7000,-76.2167,
8,Iquique,CL,75.24,clear sky,-20.2208,-70.1431,
9,Namatanai,PG,78.49,scattered clouds,-3.6667,152.4333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.count()

City                   85
Country                85
Max Temp               85
Current Description    85
Lat                    85
Lng                    85
Hotel Name             85
dtype: int64

In [10]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Katobu,ID,76.08,broken clouds,-4.9408,122.5276,La Ode Husein
1,Boyolangu,ID,75.60,overcast clouds,-8.1181,111.8935,Yos ega cv
2,Rikitea,PF,78.69,broken clouds,-23.1203,-134.9692,People ThankYou
3,Atuona,PF,78.51,broken clouds,-9.8000,-139.0333,Villa Enata
4,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Bengkulu,ID,77.09,moderate rain,-3.8004,102.2655,Grage Hotel Bengkulu
6,Hobyo,SO,78.71,overcast clouds,5.3505,48.5268,Halane Hotel and Restaurent
7,Pisco,PE,75.25,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
8,Iquique,CL,75.24,clear sky,-20.2208,-70.1431,Backpacker`s Hostel Iquique
9,Namatanai,PG,78.49,scattered clouds,-3.6667,152.4333,Seagulls Inn Namatani


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))